In [2]:
import json
import numpy as np
import pandas as pd
import requests

from settings.constants import DATA_CSV
from utils import DataLoader

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

with open('settings/specifications.json') as f:
    specifications = json.load(f)
    
info = specifications['description']
final_columns, y_column, metrics = info['final_columns'], info['y'], info['metrics']

raw_data = pd.read_csv(DATA_CSV, header=0)

loader = DataLoader()
loader.fit(raw_data)
loader.load_data()
_, val_processed = loader.split_data()
print('data: ', val_processed[:10])

val_x, val_y = val_processed[final_columns], val_processed[y_column]

req_data = {'data': json.dumps(val_x.to_dict())}
response = requests.get('http://0.0.0.0:8000/predict', data=req_data)

api_predict = response.json()['prediction']
print('predict: ', api_predict[:10])

api_score = eval(metrics)(val_y, api_predict)
print('MAPE: ', api_score)

data:        MSSubClass  LotFrontage   LotArea  Street  Alley  LotShape  LandSlope  \
1210           5     4.262680  9.325364       1      1         3          0   
558            5     4.060443  9.993008       1      1         1          0   
1095           0     4.369448  9.139703       1      1         0          0   
547            9     4.007333  8.888067       1      1         3          0   
1382           6     4.110874  8.881975       1      1         3          0   
295            8     3.637586  8.979417       1      1         0          0   
622            0     4.276666  8.862908       1      1         3          0   
480            0     4.595120  9.682467       1      1         0          0   
832            5     3.806662  9.164192       1      1         0          0   
1197           7     4.189655  9.088286       1      1         0          0   

      OverallQual  OverallCond  YearBuilt  ...  SaleType_ConLw  SaleType_New  \
1210     1.945910            4   7.597396  